In [1]:
from data.dataset import NERDataset
from models.networks import GlobalContextualDeepTransition
import pytorch_lightning as pl
from tqdm.notebook import tqdm
from torch.nn.utils.rnn import pad_packed_sequence
import re

In [2]:
sourceName = 'data/conll03/eng.train.src'
targetName = 'data/conll03/eng.train.trg'
gloveFile = 'data/conll03/trimmed.300d.Cased.txt'
symbFile = 'data/conll03/sym.glove'
data = NERDataset(sourceName, targetName, gloveFile, symbFile)
loader = data.getLoader(4096)

In [3]:
numChars = 100
charEmbedding = 128
numWords = len(data.wordIdx)
wordEmbedding = 300
contextOutputUnits = 128
contextTransitionNumber = transitionNumber = 4
encoderUnits = 256
decoderUnits = 256
prevCheckpointPath = 'lightning_logs/version_0/checkpoints/epoch=100.ckpt'
kwargs = dict(numChars=numChars, charEmbedding=charEmbedding, numWords=numWords,
                 wordEmbedding=wordEmbedding, contextOutputUnits=contextOutputUnits, contextTransitionNumber=contextTransitionNumber,
                    encoderUnits=encoderUnits, decoderUnits=decoderUnits, transitionNumber=transitionNumber, numTags=data.numTags)
model = GlobalContextualDeepTransition.load_from_checkpoint(prevCheckpointPath, **kwargs)
model.eval()
print()

In [4]:
inp = 'Harvey and Mike have authored this project'

inp = re.sub(r'[^\w\s]', '', inp)
inp = data.encodeSentence(inp)
out = model.testForward(inp)
out, lens = pad_packed_sequence(out)
out = out.view(-1).numpy()

print([data.tags[i] for i in out])

['S-PER', 'O', 'S-PER', 'O', 'O', 'O', 'O']


In [5]:
allTags = ' '.join(data.tags)
print(allTags)

B-LOC B-MISC B-ORG B-PER E-LOC E-MISC E-ORG E-PER I-LOC I-MISC I-ORG I-PER O S-LOC S-MISC S-ORG S-PER
